# BMW DCF Valuation
This notebook analyses the stock of BMW AG using a simplified DCF and multiples-based valuation.

**Goal:**
1.  Fetch financial data (Price, Income, Balance Sheet, Cash Flow).
2.  Analyze historical performance.
3.  Build a DCF model to estimate fair value.


In [1]:
# Standard Imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Local Modules
%load_ext autoreload
%autoreload 2
from ds import data, plots

# Configure Plotting Style
plots.set_style()

In [2]:
# Load All BMW Data
ticker = "BMW.DE"
peers = ["MBG.DE", "VOW3.DE", "P911.DE"]  # Mercedes, VW, Porsche

# 1. Price History
print(f"Fetching price history for {ticker}...")
df_prices = data.get_stock_data(ticker, start="2019-01-01")

# 2. Annual Financial Statements
print(f"Fetching annual financials for {ticker}...")
financials = data.get_company_financials(ticker)

# 3. Quarterly Financial Statements
print(f"Fetching quarterly financials for {ticker}...")
quarterly_financials = data.get_quarterly_financials(ticker)

# 4. Company Info & Holders
print(f"Fetching company info & holders for {ticker}...")
info = data.get_company_info(ticker)
extra_data = data.get_holders_and_recommendations(ticker)

# 5. Peer Group Data
print(f"Fetching data for peers: {peers}...")
peer_data = {}
for p in peers:
    print(f"  - {p}")
    peer_data[p] = {
        "info": data.get_company_info(p),
        "financials": data.get_company_financials(p),
        "prices": data.get_stock_data(p, start="2019-01-01"),
    }

# 6. Risk-Free Rate (Proxy: US 10Y Treasury ^TNX or German Bund if available)
print("Fetching Risk-Free Rate...")
risk_free_rate = data.get_treasury_yield("^TNX", start="2019-01-01")

print("\nData loading complete.")
print(f"Sector: {info.get('sector')}")
print(f"Beta: {info.get('beta')}")
print(f"Market Cap: {info.get('marketCap'):,.0f}")

Fetching price history for BMW.DE...
Loading BMW.DE from cache...
Fetching annual financials for BMW.DE...
Loading BMW.DE income_stmt from cache...
Loading BMW.DE balance_sheet from cache...
Loading BMW.DE cashflow from cache...
Fetching quarterly financials for BMW.DE...
Loading BMW.DE quarterly income_stmt from cache...
Loading BMW.DE quarterly balance_sheet from cache...
Loading BMW.DE quarterly cashflow from cache...
Fetching company info & holders for BMW.DE...
Loading BMW.DE info from cache...
Loading BMW.DE institutional_holders from cache...
Loading BMW.DE major_holders from cache...
Loading BMW.DE recommendations from cache...
Fetching data for peers: ['MBG.DE', 'VOW3.DE', 'P911.DE']...
  - MBG.DE


Saved MBG.DE info to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_info.json
Saved MBG.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_income_stmt.csv
Saved MBG.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_income_stmt.csv
Saved MBG.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_balance_sheet.csv
Saved MBG.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_balance_sheet.csv
Saved MBG.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_cashflow.csv
Saved MBG.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE_cashflow.csv


/workspaces/bmw-dcf-valuation-python/src/ds/data.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df: pd.DataFrame = yf.download(  # type: ignore


Saved MBG.DE to /workspaces/bmw-dcf-valuation-python/data/raw/MBG.DE.csv
  - VOW3.DE
Saved VOW3.DE info to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_info.json
Saved VOW3.DE info to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_info.json
Saved VOW3.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_income_stmt.csv
Saved VOW3.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_income_stmt.csv
Saved VOW3.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_balance_sheet.csv
Saved VOW3.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_balance_sheet.csv
Saved VOW3.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_cashflow.csv
Saved VOW3.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE_cashflow.csv


/workspaces/bmw-dcf-valuation-python/src/ds/data.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df: pd.DataFrame = yf.download(  # type: ignore


Saved VOW3.DE to /workspaces/bmw-dcf-valuation-python/data/raw/VOW3.DE.csv
  - P911.DE
Saved P911.DE info to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_info.json
Saved P911.DE info to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_info.json
Saved P911.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_income_stmt.csv
Saved P911.DE income_stmt to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_income_stmt.csv
Saved P911.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_balance_sheet.csv
Saved P911.DE balance_sheet to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_balance_sheet.csv
Saved P911.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_cashflow.csv
Saved P911.DE cashflow to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE_cashflow.csv


/workspaces/bmw-dcf-valuation-python/src/ds/data.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df: pd.DataFrame = yf.download(  # type: ignore


Saved P911.DE to /workspaces/bmw-dcf-valuation-python/data/raw/P911.DE.csv
Fetching Risk-Free Rate...


/workspaces/bmw-dcf-valuation-python/src/ds/data.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df: pd.DataFrame = yf.download(  # type: ignore


Saved ^TNX to /workspaces/bmw-dcf-valuation-python/data/raw/^TNX.csv

Data loading complete.
Sector: Consumer Cyclical
Beta: 0.774
Market Cap: 51,861,917,696


## 4. Historical Analysis
We analyze the past 4-5 years of financial performance to understand the company's trajectory.
Key metrics:
*   **Revenue Growth**: Is the company growing?
*   **EBIT Margin**: How profitable is the core business?
*   **Free Cash Flow (FCF)**: How much cash is actually generated for shareholders?


In [3]:
# --- Helper Functions ---
def get_col(df, keywords):
    """Finds a column in a DataFrame based on keywords."""
    for col in df.columns:
        if any(k in col for k in keywords):
            return df[col]
    return pd.Series(0, index=df.index)


def prepare_financial_history(financials_dict):
    """Extracts and aligns key metrics from financials dictionary."""
    income_T = financials_dict["income_stmt"].T
    balance_T = financials_dict["balance_sheet"].T
    cashflow_T = financials_dict["cashflow"].T

    df_hist = pd.DataFrame(index=income_T.index)

    # --- Income Statement ---
    df_hist["Revenue"] = get_col(income_T, ["Total Revenue", "Operating Revenue"])
    df_hist["EBIT"] = get_col(income_T, ["EBIT", "Operating Income"])
    df_hist["Pretax Income"] = get_col(income_T, ["Pretax Income"])
    df_hist["Tax Provision"] = get_col(income_T, ["Tax Provision", "Income Tax Expense"])
    df_hist["NetIncome"] = get_col(income_T, ["Net Income", "Net Income Common Stockholders"])

    # --- Balance Sheet ---
    df_hist["Total Equity"] = get_col(balance_T, ["Total Stockholder Equity", "Total Equity Gross Minority Interest"])
    df_hist["Total Debt"] = get_col(balance_T, ["Total Debt"])
    df_hist["Cash"] = get_col(balance_T, ["Cash And Cash Equivalents"])
    
    # --- Cash Flow ---
    df_hist["OCF"] = get_col(
        cashflow_T, ["Operating Cash Flow", "Total Cash From Operating Activities"]
    )
    df_hist["CapEx"] = get_col(cashflow_T, ["Capital Expenditure"])

    # --- Calculated Metrics ---
    
    # FCF (Handle CapEx sign)
    if df_hist["CapEx"].mean() > 0:
        df_hist["FCF"] = df_hist["OCF"] - df_hist["CapEx"]
    else:
        df_hist["FCF"] = df_hist["OCF"] + df_hist["CapEx"]

    # Effective Tax Rate
    df_hist["Tax Rate"] = df_hist["Tax Provision"] / df_hist["Pretax Income"]
    df_hist["Tax Rate"] = df_hist["Tax Rate"].fillna(0.25) # Fallback

    # NOPAT (Net Operating Profit After Tax)
    df_hist["NOPAT"] = df_hist["EBIT"] * (1 - df_hist["Tax Rate"])

    # Invested Capital (Simplified: Equity + Debt - Cash)
    df_hist["Invested Capital"] = df_hist["Total Equity"] + df_hist["Total Debt"] - df_hist["Cash"]

    # ROIC (Return on Invested Capital)
    df_hist["ROIC"] = df_hist["NOPAT"] / df_hist["Invested Capital"]

    # Margins
    df_hist["EBIT Margin"] = df_hist["EBIT"] / df_hist["Revenue"]
    df_hist["FCF Margin"] = df_hist["FCF"] / df_hist["Revenue"]

    return df_hist.sort_index()


# --- Prepare Data ---
df_annual = prepare_financial_history(financials)
df_quarterly = prepare_financial_history(quarterly_financials)

# Prepare Peer Data
peer_metrics = {}
for p, p_data in peer_data.items():
    peer_metrics[p] = prepare_financial_history(p_data["financials"])

print("Annual Financials (Last 5 Years):")
display(df_annual[["Revenue", "EBIT", "NetIncome", "FCF", "ROIC"]].tail().style.format("{:,.0f}"))

Annual Financials (Last 5 Years):


,Revenue,EBIT,NetIncome,FCF,ROIC
2020-12-31 00:00:00,nan,nan,nan,nan,nan
2021-12-31 00:00:00,"111,239,000,000","22,287,000,000","12,382,000,000","9,295,000,000",0
2022-12-31 00:00:00,"142,610,000,000","31,362,000,000","17,941,000,000","14,473,000,000",0
2023-12-31 00:00:00,"155,498,000,000","27,963,000,000","11,290,000,000","6,661,000,000",0
2024-12-31 00:00:00,"142,380,000,000","20,783,000,000","7,290,000,000","-4,639,000,000",0


In [4]:
# Prepare data for plotting (Fixing Pylance type inference)
# Explicitly convert index to DatetimeIndex and extract years
df_annual.index = pd.to_datetime(df_annual.index)
years = pd.DatetimeIndex(df_annual.index).year


In [5]:
# --- Visualization: Comprehensive Dashboard ---
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=(
        "Revenue & EBIT Margin (Annual)",
        "Free Cash Flow Composition (Annual)",
        "Quarterly Revenue Trend",
        "Stock Price History",
        "ROIC History (Return on Invested Capital)",
        "Peer Comparison: EBIT Margin",
    ),
    specs=[
        [{"secondary_y": True}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
    ],
    vertical_spacing=0.10,
)

# 1. Revenue & EBIT Margin (Annual)
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["Revenue"],
        name="Annual Revenue",
        marker_color="rgb(55, 83, 109)",
    ),
    row=1,
    col=1,
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["EBIT Margin"],
        name="EBIT Margin",
        mode="lines+markers",
        line=dict(color="rgb(26, 118, 255)", width=3),
    ),
    row=1,
    col=1,
    secondary_y=True,
)

# 2. FCF Composition (Annual)
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["OCF"],
        name="Operating Cash Flow",
        marker_color="green",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["CapEx"],
        name="CapEx",
        marker_color="crimson",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["FCF"],
        name="Free Cash Flow",
        line=dict(color="black", width=3, dash="dash"),
    ),
    row=1,
    col=2,
)

# 3. Quarterly Revenue Trend
fig.add_trace(
    go.Scatter(
        x=df_quarterly.index,
        y=df_quarterly["Revenue"],
        name="Quarterly Revenue",
        fill="tozeroy",
        line=dict(color="purple"),
    ),
    row=2,
    col=1,
)

# 4. Stock Price History
fig.add_trace(
    go.Scatter(
        x=df_prices.index,
        y=df_prices["Close"],
        name="Stock Price",
        line=dict(color="orange"),
    ),
    row=2,
    col=2,
)

# 5. ROIC History
fig.add_trace(
    go.Bar(
        x=years,
        y=df_annual["ROIC"],
        name="ROIC",
        marker_color="teal",
    ),
    row=3,
    col=1,
)

# 6. Peer Comparison: EBIT Margin
# Add BMW first
fig.add_trace(
    go.Scatter(
        x=years,
        y=df_annual["EBIT Margin"],
        name=f"{ticker} Margin",
        mode="lines+markers",
        line=dict(width=3),
    ),
    row=3,
    col=2,
)
# Add Peers
for p, p_df in peer_metrics.items():
    # Align peer years
    p_years = pd.DatetimeIndex(p_df.index).year
    fig.add_trace(
        go.Scatter(
            x=p_years,
            y=p_df["EBIT Margin"],
            name=f"{p} Margin",
            mode="lines",
            line=dict(dash="dot"),
        ),
        row=3,
        col=2,
    )

# Layout Updates
fig.update_layout(
    height=1200,
    title_text=f"Financial Overview: {ticker} vs Peers",
    template="plotly_white",
    showlegend=True,
)
fig.update_yaxes(title_text="Revenue", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Margin", tickformat=".1%", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Amount", row=1, col=2)
fig.update_yaxes(title_text="ROIC", tickformat=".1%", row=3, col=1)
fig.update_yaxes(title_text="EBIT Margin", tickformat=".1%", row=3, col=2)

fig.show()